In [ ]:
CREATE VOLATILE TABLE result_table, MULTISET AS (
    SELECT customer_id, 
           SUM(order_amount) as total_spent
    FROM orders 
    WHERE order_date >= DATE '2024-01-01'
    GROUP BY customer_id
) WITH DATA
PRIMARY INDEX (customer_id)
ON COMMIT PRESERVE ROWS;


In [ ]:
# SELECT TO_CHAR(timestamp_column, 'YYYY-MM-DD HH:MI:SS') FROM table_name;


### Pandas Fastload

In [ ]:
import teradatasql
import pandas as pd

# Create sample DataFrame
df = pd.DataFrame({
    'Hash_Id': ['value1', 'value2', 'value3'],
    'Clean_Claim': ['text1', 'text2', 'text3'],
    'FTC_Label': ['data1', 'data2', 'data3']
})

# Connection parameters
conn_params = {
    'host': 'your_host',
    'user': 'your_user',
    'password': 'your_password'
}

create_tables_sql = """
CREATE MULTISET TABLE table_name (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_1 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_2 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);
"""

with teradatasql.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        # Create tables
        for stmt in create_tables_sql.split(';'):
            if stmt.strip():
                cur.execute(stmt)
        
        # FastLoad setup
        cur.execute("SESSIONS 8")
        cur.execute("ERRLIMIT 50")
        cur.execute("""
            BEGIN LOADING table_name
            ERRORFILES error_table_1, error_table_2;
        """)
        cur.execute("SET RECORD VARTEXT ','")
        cur.execute("""
            DEFINE Hash_Id (CHAR(128)),
                   Clean_Claim (VARCHAR(80)),
                   FTC_Label (VARCHAR(20))
        """)
        cur.execute("""
            INSERT INTO table_name (Hash_Id, Clean_Claim, FTC_Label)
            VALUES (:Hash_Id, :Clean_Claim, :FTC_Label)
        """)
        
        # Load data from DataFrame using plain tuples
        for row in df.itertuples(index=False, name=None):
            cur.execute("INSERT INTO table_name VALUES (?, ?, ?)", row)
        
        # End loading
        cur.execute("END LOADING")


### Pandas FastLoad Version 2

In [ ]:
import teradatasql
import pandas as pd

# Create sample DataFrame
df = pd.DataFrame({
    'Hash_Id': ['value1', 'value2', 'value3'],
    'Clean_Claim': ['text1', 'text2', 'text3'],
    'FTC_Label': ['data1', 'data2', 'data3']
})

# Connection parameters
conn_params = {
    'host': 'your_host',
    'user': 'your_user',
    'password': 'your_password'
}

create_tables_sql = """
CREATE MULTISET TABLE table_name (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_1 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_2 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);
"""

with teradatasql.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        # Create tables
        for stmt in create_tables_sql.split(';'):
            if stmt.strip():
                cur.execute(stmt)
        
        # Initialize FastLoad
        fastload = conn.fastload()
        
        # Configure FastLoad
        fastload.sessions(8)
        fastload.error_limit(50)
        fastload.error_tables('error_table_1', 'error_table_2')
        
        # Define target table and columns
        fastload.target('table_name', ['Hash_Id', 'Clean_Claim', 'FTC_Label'])
        
        # Load data from DataFrame using plain tuples
        for row in df.itertuples(index=False, name=None):
            fastload.put(row)
        
        # Complete the load
        fastload.end()


### Polars FastLoad

In [ ]:
import teradatasql
import polars as pl

# Create sample DataFrame
df = pl.DataFrame({
    'Hash_Id': ['value1', 'value2', 'value3'],
    'Clean_Claim': ['text1', 'text2', 'text3'],
    'FTC_Label': ['data1', 'data2', 'data3']
})

# Connection parameters
conn_params = {
    'host': 'your_host',
    'user': 'your_user',
    'password': 'your_password'
}

create_tables_sql = """
CREATE MULTISET TABLE table_name (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_1 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_2 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);
"""

with teradatasql.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        # Create tables
        for stmt in create_tables_sql.split(';'):
            if stmt.strip():
                cur.execute(stmt)
        
        # FastLoad setup
        cur.execute("SESSIONS 8")
        cur.execute("ERRLIMIT 50")
        cur.execute("""
            BEGIN LOADING table_name
            ERRORFILES error_table_1, error_table_2;
        """)
        cur.execute("SET RECORD VARTEXT ','")
        cur.execute("""
            DEFINE Hash_Id (CHAR(128)),
                   Clean_Claim (VARCHAR(80)),
                   FTC_Label (VARCHAR(20))
        """)
        cur.execute("""
            INSERT INTO table_name (Hash_Id, Clean_Claim, FTC_Label)
            VALUES (:Hash_Id, :Clean_Claim, :FTC_Label)
        """)
        
        # Load data from DataFrame using Polars iteration
        for row in df.iter_rows():
            cur.execute("INSERT INTO table_name VALUES (?, ?, ?)", row)
        
        # End loading
        cur.execute("END LOADING")


### FastLoad Version2

In [ ]:
import teradatasql
import polars as pl

# Create sample DataFrame
df = pl.DataFrame({
    'Hash_Id': ['value1', 'value2', 'value3'],
    'Clean_Claim': ['text1', 'text2', 'text3'],
    'FTC_Label': ['data1', 'data2', 'data3']
})

# Connection parameters
conn_params = {
    'host': 'your_host',
    'user': 'your_user',
    'password': 'your_password'
}

create_tables_sql = """
CREATE MULTISET TABLE table_name (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_1 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_2 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);
"""

with teradatasql.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        # Create tables
        for stmt in create_tables_sql.split(';'):
            if stmt.strip():
                cur.execute(stmt)
        
        # Initialize FastLoad
        fastload = conn.fastload()
        
        # Configure FastLoad
        fastload.sessions(8)
        fastload.error_limit(50)
        fastload.error_tables('error_table_1', 'error_table_2')
        
        # Define target table and columns
        fastload.target('table_name', ['Hash_Id', 'Clean_Claim', 'FTC_Label'])
        
        # Load data from DataFrame
        for row in df.iter_rows():
            fastload.put(row)
        
        # Complete the load
        fastload.end()


### FastLoad From File

In [ ]:
import teradatasql

# Connection parameters
conn_params = {
    'host': 'your_host',
    'user': 'your_user',
    'password': 'your_password'
}

create_tables_sql = """
CREATE MULTISET TABLE table_name (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_1 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_2 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);
"""

with teradatasql.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        # Create tables
        for stmt in create_tables_sql.split(';'):
            if stmt.strip():
                cur.execute(stmt)
        
        # Initialize FastLoad
        fastload = conn.fastload()
        
        # Configure FastLoad
        fastload.sessions(8)
        fastload.error_limit(50)
        fastload.error_tables('error_table_1', 'error_table_2')
        
        # Define target table and columns
        fastload.target('table_name', ['Hash_Id', 'Clean_Claim', 'FTC_Label'])
        
        # Read and load directly from file
        with open('data.csv', 'r') as file:
            next(file)  # Skip header
            for line in file:
                fields = line.strip().split(',')
                fastload.put(fields)
        
        # Complete the load
        fastload.end()


### Pyarrow Fastload from Parquet File

In [ ]:
import teradatasql # 17.20.0.0
import pyarrow.parquet as pq # 14.0.1

# Connection parameters
conn_params = {
    'host': 'your_host',
    'user': 'your_user',
    'password': 'your_password'
}

create_tables_sql = """
CREATE MULTISET TABLE table_name (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_1 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);

CREATE MULTISET TABLE error_table_2 (
    Hash_Id CHAR(128),
    Clean_Claim VARCHAR(80),
    FTC_Label VARCHAR(20),
    error_code INTEGER,
    error_desc VARCHAR(256)
) PRIMARY INDEX (Hash_Id);
"""

with teradatasql.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        # Create tables
        for stmt in create_tables_sql.split(';'):
            if stmt.strip():
                cur.execute(stmt)
        
        # Initialize FastLoad
        fastload = conn.fastload()
        
        # Configure FastLoad
        fastload.sessions(8)
        fastload.error_limit(50)
        fastload.error_tables('error_table_1', 'error_table_2')
        
        # Define target table and columns
        fastload.target('table_name', ['Hash_Id', 'Clean_Claim', 'FTC_Label'])
        
        # Read and load directly from parquet file
        parquet_file = pq.ParquetFile('data.parquet')
        for batch in parquet_file.iter_batches():
            for row in batch.to_pylist():
                fastload.put([row['Hash_Id'], row['Clean_Claim'], row['FTC_Label']])
        
        # Complete the load
        fastload.end()
